In [1]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.features._version import VersionManager

## Branch Versioning and Parcel Fabrics with the ArcGIS API for Python
The ArcGIS API for Python (Python API) is a powerful, modern Pythonic library that supports the latest releases of ArcGIS Enterprise and ArcGIS Online and provides a consistent programmatic experience for scripting and automating across the ArcGIS product suite.

Think of the Python API as programmatic access to the `VersionManagementServer` and `ParcelFabricServer` REST endpoints. All functions found in these enpoints are wrapped with functions in Python API.

In [2]:
gis = GIS("https://dev0016752.esri.com/portal/", "admin", "esri.agp", verify_cert=False)
vms_url = "https://dev0016752.esri.com/server/rest/services/SlackStreet/VersionManagementServer"

# Get the VersionManagementServer endpoint and version name
vms = VersionManager(vms_url, gis)
    
for v in vms.all:
    print(v.properties.versionName)

Setting `verify_cert` to False is a security risk, use at your own risk.


sde.DEFAULT
ADMIN.SplitCOGO_MVVM
ADMIN.A
ADMIN.B
ADMIN.C


## Versioning Conflicts
### Reconcile and Post all Versions

In this scenario, all editing by all users in all versions has taken place. No reconcile and post has occurred at the end of any edit session. The code in the cell below will reconcile and attempt to post all versions alphabetically (`vms.all` is a `list[]`)

The code will detect conflicts and report the features in conflict. 

**WARNING** 
Reconciling a version a second time will automatically resolve conflicts in favor of the edit version. 
[Manage Branch Versions](https://pro.arcgis.com/en/pro-app/latest/help/data/geodatabases/overview/manage-branch-version-conflicts.htm#GUID-0F4BAA85-E1B8-4736-A80D-0966B0A400BD)

In [ ]:
for v in vms.all:
    if not v.properties.versionName == "sde.DEFAULT":
        version_name = v.properties.versionName
        with vms.get(version_name, "read") as version:
            print("Reconciling", version_name)
            version.mode = "edit"
            rec = version.reconcile(
                end_with_conflict=False,
                with_post=True,
                conflict_detection="byObject"
            )
            if rec["hasConflicts"]:
                print("\tCONFLICTS DETECTED IN", version_name)
                conflicts = version.conflicts()
                print(conflicts)

### Repeatability

It is possible to repeat the conflicts query after the intial reconcile. 

In [5]:
for v in vms.all:
    if not v.properties.versionName == "sde.DEFAULT":
        version_name = v.properties.versionName
        with vms.get(version_name, "read") as version:
            print(version_name, "conflicts found:", len(version.conflicts()["conflicts"]))

ADMIN.SplitCOGO_MVVM conflicts found: 0
ADMIN.A conflicts found: 0
ADMIN.B conflicts found: 2
ADMIN.C conflicts found: 0


### Show the conflict features
Display the attributes of the current version, common ancestor and default version.

For example, show the conflict features of a polygon parcel type layer (layerID 24)

In [6]:
version_name = "ADMIN.B"

with vms.get(version_name, "read") as version:
    conflicts = version.conflicts()
    parcel_conflicts = [c for c in conflicts["conflicts"] if c["layerId"] == 24]
parcel_conflicts[0]["updateUpdateConflicts"]

[{'branchVersion': {'attributes': {'OBJECTID': 401,
    'Name': None,
    'CreatedByRecord': '{70B01D05-ADE3-41D2-9525-98C17905EF4F}',
    'RetiredByRecord': '{AF030208-E25C-4CA0-8F4A-97F87A4CB844}',
    'StatedArea': None,
    'StatedAreaUnit': None,
    'CalculatedArea': None,
    'MiscloseRatio': None,
    'MiscloseDistance': None,
    'IsSeed': 0,
    'created_user': 'admin',
    'created_date': 1701287035000,
    'last_edited_user': 'admin',
    'last_edited_date': 1701287669000,
    'GlobalID': '{D23ECF18-C127-468D-A563-B17D3026B1C9}',
    'VALIDATIONSTATUS': 2,
    'Shape.STArea()': 1534.223587109188,
    'Shape.STLength()': 530.5539629976115},
   'geometry': {'hasZ': True,
    'rings': [[[6318709.967443049, 1928731.34921439, 0],
      [6318969.282541215, 1928727.7051928043, 0],
      [6318969.586346388, 1928721.3285651505, 0],
      [6318710.271248221, 1928725.8833460659, 0],
      [6318709.967443049, 1928731.34921439, 0]]]}},
  'ancestorVersion': {'attributes': {'OBJECTID': 40

## Version Differences
Show all changes between a branch version and the Default version

In [8]:
# Get the OID and CBR of the features edited in branch.
# Open the version and start an edit session
with vms.get(version_name, "read") as version:
    version.mode = "edit"
    
    # Get version differences for lines and parcels of ROW parcel type
    diff = version.differences(result_type="features", layers=[23, 24])
    
    # Get all recent changes that differ between branch version and Default
    parcel_differences = [f for f in diff["features"] if f["layerId"] == 24][0]
    inserts = [f["attributes"] for f in parcel_differences["inserts"]]
    updates = [f["attributes"] for f in parcel_differences["updates"]]
    all_edits = inserts + updates
    
all_edits

[{'OBJECTID': 1202,
  'Name': None,
  'CreatedByRecord': '{AF030208-E25C-4CA0-8F4A-97F87A4CB844}',
  'RetiredByRecord': None,
  'StatedArea': None,
  'StatedAreaUnit': None,
  'CalculatedArea': None,
  'MiscloseRatio': None,
  'MiscloseDistance': None,
  'IsSeed': 0,
  'created_user': 'admin',
  'created_date': 1701287660000,
  'last_edited_user': 'admin',
  'last_edited_date': 1701287669000,
  'GlobalID': '{73941D37-5BB9-472E-8B54-001BC5384C2F}',
  'VALIDATIONSTATUS': 2,
  'Shape.STArea()': 92.48489835622219,
  'Shape.STLength()': 64.11570515889085},
 {'OBJECTID': 1303,
  'Name': None,
  'CreatedByRecord': '{AF030208-E25C-4CA0-8F4A-97F87A4CB844}',
  'RetiredByRecord': None,
  'StatedArea': None,
  'StatedAreaUnit': None,
  'CalculatedArea': None,
  'MiscloseRatio': None,
  'MiscloseDistance': None,
  'IsSeed': 0,
  'created_user': 'admin',
  'created_date': 1701287660000,
  'last_edited_user': 'admin',
  'last_edited_date': 1701287669000,
  'GlobalID': '{4DFEB76B-9A9A-49D8-B100-66F685

### Detect Conflicting Parcel Fabric Edits

In this case, long right-of-way polygons could be edited in multiple versions. The first version to be posted wins and subsequent posts to default will be in conflict. In the parcel fabric, we may be able to detect which features have been edited using the `RetiredByRecord` field.

Using the `differences` function on the `version` object, it is possible to filter the features by having a `RetiredByRecord` field.

In [10]:
edited_parcel_features = [{"objectid":f["OBJECTID"], "retiredbyrecord": f["RetiredByRecord"]} for f in all_edits]
edited_parcel_features

[{'objectid': 1202, 'retiredbyrecord': None},
 {'objectid': 1303, 'retiredbyrecord': None},
 {'objectid': 401,
  'retiredbyrecord': '{AF030208-E25C-4CA0-8F4A-97F87A4CB844}'}]